# PART 1 : SEGMENTATION + NOISE ADD

In [1]:
import os
import sys
import copy
from matplotlib import pyplot as plt

%matplotlib inline

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import data_augmentation
from NN_segmentation.tst_dataset import cardiacdata
from NN_segmentation.UNET3D_D4 import UNet3D
from NN_segmentation.Auto3D_D4 import Auto3D
import numpy as np

In [2]:
# LOAD DATA
path_data = r"/home/jovyan/project_data/ACDC-Daten"
add_noise = True
single_noise_probability = 50

data_list = []
label_list = []

for root, _, files in os.walk(path_data):
    if os.path.isfile(os.path.join(root, "image.nii.gz")):
        try:
            data_list.append(cardiacdata(img_dir=os.path.join(root, "image.nii.gz"), label_dir=os.path.join(root, "label.nii.gz")))
        except:
            pass
        
data_noise = []#copy.deepcopy(data_list)
for i in range(len(data_list)):
    noisy_data = []
    for j in range(data_list[i].img.shape[0]):
        for k in range(data_list[i].img.shape[1]):
            noisy_data.append(data_augmentation.augmentation(np.squeeze(data_list[i].img[j,k,:,:]), single_noise_probability))
    data_noise.append(np.array(noisy_data).reshape(data_list[i].img.shape))
    
data_noisy_list = copy.deepcopy(data_list)
for i in range(len(data_list)):
    data_noisy_list[i].img = data_noise[i]

In [3]:
#for num in range(len(data_list)):
#    fig, axes = plt.subplots(1, 2)
#    axes[0].imshow(np.squeeze(data_list[num].img[0,0,:,:]), cmap="gray")
#    axes[0].axis("off")
#    axes[1].imshow(np.squeeze(data_noisy_list[num].img[0,0,:,:]), cmap="gray")
#    axes[1].axis("off")
#    plt.show()

In [4]:
# RUN SEGMENTATION
from torch.utils.data import DataLoader
import torch

MDL_PATH=r"/home/jovyan/project_segmentation_models/19"
data = data_list
#data = data_noisy_list


# network
net = Auto3D(num_classes=4, in_channels=1, depth=4, start_filts=32, res=False).cuda()
#net = Auto3D(num_classes=4, depth=4, start_filts=32, in_channels=1, res=True).cuda()

net.load_state_dict(torch.load(os.path.join(MDL_PATH, "model_best.pth.tar"))['state_dict'])

normOrg_list=[]
normGT_list=[]
normSeg_list=[]
normError_list = []

for i in range(len(data)):
    tst_loader = DataLoader(data[i], batch_size=1, shuffle=False, num_workers=0)
    img_shape = data[i].img.shape
    normOrg=np.zeros((1,img_shape[1],img_shape[2],img_shape[3]),dtype=np.float32)
    normGT=np.zeros((1,img_shape[1],img_shape[2],img_shape[3]),dtype=np.int16)
    normSeg=np.zeros((1,img_shape[1],img_shape[2],img_shape[3]),dtype=np.int16)
    #dice = np.zeros((1, 3))
    net.eval()
    for step, (img, seg_gt) in enumerate(tst_loader, 0):
        img, seg_gt = img.cuda(), seg_gt.cuda()
        pred = net(img)
        _, pred = torch.max(pred, 1)
        #pred, _ = torch.max(pred, 1)

        pred = pred.squeeze().detach().cpu().numpy().astype(np.int8)
        img = img.squeeze().detach().cpu().numpy()
        gt = seg_gt.squeeze().detach().cpu().numpy().astype(np.int8)
        #for j in range(3):
        #    dice[step, j] = dice_comp(pred==j+1, gt==j+1)
        normOrg[step]=img
        normGT[step]=gt
        normSeg[step]=pred

    normOrg=np.reshape(normOrg,[int(normOrg.shape[1]/8),8,144,144])
    normGT=np.reshape(normGT,[int(normGT.shape[1]/8),8,144,144])
    normSeg=np.reshape(normSeg,[int(normSeg.shape[1]/8),8,144,144])
    normError=np.abs(normGT.astype(np.float32)-normSeg.astype(np.float32))
    normOrg=normOrg-normOrg.min()
    
    normOrg_list.append(normOrg)
    normGT_list.append(normGT)
    normSeg_list.append(normSeg)
    normError_list.append(normError)


#%% Display the output images
index = 0
for dispind in range(normOrg[index].shape[0]):
    plot= lambda x: plt.imshow(x,cmap=plt.cm.gray,interpolation='bilinear')
    plot1= lambda x: plt.imshow(x,interpolation='bilinear')
    plt.clf()
    plt.subplot(141)
    plot(np.abs(normOrg[index][dispind,vol_slice,:,:]))
    plt.axis('off')
    plt.title('Original')
    plt.subplot(142)
    plot1(np.abs(normGT[index][dispind,vol_slice,:,:]))
    plt.axis('off')
    plt.title('True labels')
    plt.subplot(143)
    plot1(np.abs(normSeg[index][dispind,vol_slice,:,:]))
    plt.axis('off')
    plt.title('Segmentation')
    plt.subplot(144)
    plot(np.abs(normError[index][dispind,vol_slice,:,:]))
    plt.title('Error')
    plt.axis('off')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0,wspace=.01)
    plt.show()
# purple=0, blue=1, green=2, yellow=3

RuntimeError: CUDA out of memory. Tried to allocate 304.00 MiB (GPU 0; 7.80 GiB total capacity; 6.71 GiB already allocated; 59.44 MiB free; 6.82 GiB reserved in total by PyTorch)